# PSTAT 134
## Assignment 3
## Spring 2018
## J Steven Raquel

###  Problem 1



In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess as sp
import pickle
import sklearn.decomposition as skld

import helper_basketball as h
import imp
imp.reload(h);

In [ ]:
allshots = pickle.load(open('allshots2016-17.pkl', 'rb'))
X = pickle.load(open('allpatterns2016-17.pkl', 'rb'))

# allmade = allshots.loc[allshots.SHOT_MADE_FLAG==1]
allmade = allshots
allmade.head()

In [ ]:
## players info
player_ids = allmade.PlayerID.unique()
num_players = player_ids.size

## bin edge definitions in inches
xedges = (np.linspace(start=-25, stop=25, num=151, dtype=np.float)) * 12
yedges = (np.linspace(start= -4, stop=31, num=106, dtype=np.float)) * 12

## number of bins is one less than number of edges
nx = xedges.size - 1
ny = yedges.size - 1

## 2d histogram containers for binned counts and smoothed binned counts
all_counts = {}
all_smooth = {}

## data matrix: players (row) by vectorized 2-d court locations (column)
for i, one in enumerate(allmade.groupby('PlayerID')):
    
    ## what does this line do?
    pid, pdf = one
    
    ## h.bin_shots: what is this function doing?
    tmp1, xedges, yedges = h.bin_shots(pdf, bin_edges=(xedges, yedges), density=True, sigma=2)
    tmp2, xedges, yedges = h.bin_shots(pdf, bin_edges=(xedges, yedges), density=False)
    
    ## vectorize and store into dictionary
    all_smooth[pid] = tmp1.reshape(-1)
    all_counts[pid] = tmp2.reshape(-1)

In [ ]:
## Non-negative Matrix Factorization
# X = np.stack(all_smooth.values()).T

## what are the different options mean for NMF()?
model = skld.NMF(n_components=10, init='nndsvda', max_iter=500, random_state=0)
W = model.fit_transform(X)
H = model.components_

# visulizing the bases with 10 components
fig, ax = plt.subplots(5, 2, figsize=(20,40))

for i, axi in enumerate(ax.flatten()):
    h.plot_shotchart(W[:,i], xedges, yedges, ax=axi)
    axi.set_title('NMF component ' + str(i))

In [ ]:
## what are the different options mean for NMF()?
model = skld.NMF(n_components=5, init='nndsvda', max_iter=500, random_state=0)
W = model.fit_transform(X)
H = model.components_

# visualizing the bases with 6 components
fig, ax = plt.subplots(5, 2, figsize=(20,40))

for i, axi in enumerate(ax.flatten()):
    h.plot_shotchart(W[:,i], xedges, yedges, ax=axi)
    axi.set_title('NMF component ' + str(i))

In [ ]:
## what are the different options mean for NMF()?
model = skld.NMF(n_components=15, init='nndsvda', max_iter=500, random_state=0)
W = model.fit_transform(X)
H = model.components_

# visualizing the bases with 15 components
fig, ax = plt.subplots(5, 3, figsize=(20,40))

for i, axi in enumerate(ax.flatten()):
    h.plot_shotchart(W[:,i], xedges, yedges, ax=axi)
    axi.set_title('NMF component ' + str(i))